Todo esto quedó afuera del notebook 7

### Desigualdades válidas bajo demanda

In [1]:
def construir_modelo_pool(G, C):
    modelo = construir_modelo_col(G, C)
    X, W = recuperar_variables(G, C, modelo)
    for Q in nx.find_cliques(G):
        for k in C:
            modelo.add_user_cut_constraint(modelo.sum(X[v][k] for v in Q) <= W[k])
    return modelo

In [2]:
modelo_pool = construir_modelo_pool(G, C)
modelo_pool.parameters.preprocessing.presolve = 0
resolver(modelo_pool)

NameError: name 'G' is not defined

### Rankear experimentalmente desigualdades válidas


Tenemos una desigualdad válida y queremos ver si corta soluciones fraccionarias. Por ejemplo, podría pasar que esté implicada, es decir, que se obtenga a partir de desigualdades del modelo. Maximizar el lado izquierdo de la desigualdad.

In [3]:
relajacion2 = relajacion_lineal(modelo)
x_0_0 = relajacion2.get_var_by_name("x_0_0")
x_1_0 = relajacion2.get_var_by_name("x_1_0")
x_2_0 = relajacion2.get_var_by_name("x_2_0")
relajacion2.maximize(x_0_0 + x_1_0 + x_2_0)
sol = resolver(relajacion2)
print(relajacion2.objective_value)

NameError: name 'relajacion_lineal' is not defined

## Eliminación de soluciones simétricas

Otra forma de reducir el tiempo de optimización es reducir el espacio de búsqueda, durante la exploración de nodos del árbol B&B.
Decimos que un conjunto de soluciones $s_1$, $s_2$, ..., $s_n$ son simétricas si tienen el mismo valor objetivo. Observemos que podemos eliminar $s_2$, ..., $s_n$ sin afectar el resultado de la optimización. En el caso del problema de coloreo, esto equivale a eliminar soluciones con misma cantidad de colores.

Para eliminarlas, incorporamos nuevas restricciones a la formulación. Por ejemplo, en COL habíamos incorporado $(COL.3)$ para eliminar ciertas soluciones innecesarias. 

Ahora vamos a considerar distintas familias de desigualdades que eliminan soluciones simétricas, propuestas por I.$~$Méndez-Díaz y P.$~$Zavala. Supongamos que los colores están enumerados $\mathcal{C}=1,\ldots,c$:
- Orden en la cantidad de colores: Claramente una solución que utilice los colores de un subconjunto de tamaño $k$  de $\mathcal{C}$ usa la misma cantidad de colores que otra a la que se le asignan colores de $1,\ldots,k$ (por ejemplo, para una solución que utilice los colores $3,5,8$ existe otra que utiliza $1,2,3$). Podemos eliminar estas soluciones adicionales agregando:
$$w_k \geq w_{k+1},~~ \forall~ k = 1,\ldots,c-1~~~~~~(S1)$$
- Orden en los cardinales de los colores: Observemos que, para una solución que utiliza los colores $1,\ldots,k$, existen otras $k!-1$ soluciones simétricas en donde se hizo una permutación de sus colores. Podemos eliminar parcialmente estas soluciones si imponemos que los primeros colores tengan mayor cardinal que los últimos. Es decir, si $V_i$ representa a aquellos vértices pintados de $i$, podemos restringirnos a soluciones donde $|V_1| \geq |V_2| \geq ... \geq |V_k|$ con:
$$\sum_{v \in V} x_{vk} \geq \sum_{v \in V} x_{vk+1},~~ \forall~ k = 1,\ldots,c-1~~~~~~(S2)$$
- Eliminación de indistinguibles I: Teniendo en cuenta una enumeración de los vértices ($v_1$, $v_2$, ...) podemos, sin perder generalidad, imponer que el primer vértice sea coloreado con el primer color. Luego $v_2$ puede, o bien utilizar el mismo color, o bien uno nuevo, por lo que le permitimos colorearse con colores 1 o 2. En otras palabras, $v_1$ no se puede colorear con $\{2,\ldots,c\}$, $v_2$ no lo puede hacer con $\{3,\ldots,c\}$, etc. Siguiendo esta lógica proponemos:
$$x_{v_i k} = 0,~~ \forall~ k=i+1,\ldots,c,~ \forall~i=1,\ldots,c-1~~~~~~(S3)$$
Estas restricciones no son compatibles con $(S2)$.
- Eliminación de indistinguibles II: Dejamos al lector entender el significado de estas desigualdades que, junto con $(S1)$ y $(S3)$, eliminan todas las $k!-1$ soluciones simétricas:
$$\sum_{j=k-1}^{i-1} x_{u_jk-1} \geq x_{v_ik}~~ \forall~ i=k+1,\ldots,c,~ \forall~k=2,\ldots,c-1~~~~~~(S4)$$

### Actividad 4

- Incorpore las restricciones que eliminan simetrías a la formulación. Llamémoslas:
  - $COL_1$: Formulación original
  - $COL_2$: $COL_1$ + $(S1)$ (Orden en la cantidad de colores)
  - $COL_3$: $COL_2$ + $(S2)$ (Orden en los cardinales de los colores)
  - $COL_4$: $COL_2$ + $(S3)$ (Eliminación de indistinguibles I)
  - $COL_5$: $COL_4$ + $(S4)$ (Eliminación de indistinguibles II)
- Para grafos con densidades de 10%, 50% y 90%, ¿hasta qué cantidad de vértices es posible resolver una instancia con las formulaciones dadas (por densidad) para un tiempo límite de 5 minutos? Concluya cuáles de las restricciones que eliminan simetrías conviene considerar.